# Homework 1: PCA

Problem 1 - Principal Component Analysis
---

In this problem you'll be implementing Dimensionality reduction using Principal Component Analysis technique. 

The gist of PCA Algorithm to compute principal components is follows:
- Calculate the covariance matrix X of data points.
- Calculate eigenvectors and corresponding eigenvalues.
- Sort the eigenvectors according to their eigenvalues in decreasing order.
- Choose first k eigenvectors which satisfies target explained variance.
- Transform the original data of shape m observations times n features into m observations times k selected features.


The skeleton for the *PCA* class is below. Scroll down to find more information about your tasks.

In [1]:
import math
import pickle
import gzip
import numpy as np
import pandas
import matplotlib.pylab as plt
%matplotlib inline
from numpy import linalg as LA

In [2]:
 from sklearn.preprocessing import StandardScaler
    
class PCA:
    def __init__(self, target_explained_variance=None):
        """
        explained_variance: float, the target level of explained variance
        """
        self.target_explained_variance = target_explained_variance
        self.feature_size = -1

    def standardize(self, X):
        """
        standardize features using standard scaler
        :param X: input data with shape m (# of observations) X n (# of features)
        :return: standardized features (Hint: use skleanr's StandardScaler.)
        """
        # Q1. Standardize X using sklearn's StandardScaler. Read the documentation's example. https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html#sklearn.preprocessing.StandardScaler
        # Hint: In the example, they used .fit() and .transform() methods separately. You can use another method that has both fit and transform.
        # YOUR CODE HERE
        std = StandardScaler()
        
        return std.fit_transform(X)

    def compute_mean_vector(self, X_std):
        """
        compute mean vector
        :param X_std: transformed data
        :return mean vector of shape (# of features,)
        """
        # Q2. return mean vector of shape (# of features,). Hint: averaging over the rows for each column. 
       
        mean_vector = np.mean(X_std, axis=0)  
    
        return mean_vector

    def compute_cov(self, X_std, mean_vec):
        """
        Covariance using mean, (don't use any numpy.cov)
        :param X_std:
        :param mean_vec:
        :return n X n matrix:: covariance matrix
        
        :return 784 X 784 matrix
        """
        # Q3. Caculate covariance matrix https://en.wikipedia.org/wiki/Covariance_matrix
        # Hint: E[(X-mu)T(X-mu)]. You can assume equal probability when calculating the expected value.
        cov = np.dot((X_std - mean_vec).T, (X_std-mean_vec)) / (X_std.shape[0] - 1)
        
        return cov

    def compute_eigen_vector(self, cov_mat):
        """
        Eigenvector and eigen values using numpy. Uses numpy's eigenvalue function
        :param cov_mat:
        :return: (eigen_values, eigen_vector)
        """
        # Q4. Return eigenvalues and engenvectors. 
        # Hint: Use appropriate function in linalg package. https://numpy.org/doc/stable/reference/routines.linalg.html
        # YOUR CODE HERE
        e_vals, e_vector = LA.eig(cov_mat)
        return (e_vals, e_vector)

    def compute_explained_variance(self, eigen_vals):
        """
        Q5. Sort eigen values and compute explained variance ratio.
        explained variance informs the amount of information (variance)
        can be attributed to each of  the principal components.
        :param eigen_vals:
        :return: explained variance ratio.
        """
        
        idxs = np.argsort(eigen_vals)[::-1]
        eigen_vals = eigen_vals[idxs]
        
        total = eigen_vals.sum()
        e_var_ratio = eigen_vals / total
        return e_var_ratio

    def cumulative_sum(self, var_exp):
        """
        return cumulative sum of explained variance. 
        :param var_exp: explained variance ratio
        :return: cumulative explained variance ratio
        """
        return np.cumsum(var_exp)

    def compute_weight_matrix(self, eig_pairs, cum_var_exp):
        """
        compute weight matrix of top principal components conditioned on target
        explained variance.
        (Hint : use cumilative explained variance ratio and target_explained_variance to find
        top components)
        
        :param eig_pairs: list of tuples containing eigenvalues and eigenvectors, 
        sorted by eigenvalues in descending order (the biggest eigenvalue and corresponding eigenvectors first).
        :param cum_var_exp: cumulative expalined variance by features
        :return: weight matrix (the shape of the weight matrix is n X k)
        """
        matrix_w = np.ones((self.feature_size, 1)) 
        # Q6. In this function, you will implement weight matrix calculation.
        # For each iteration, check the cumulative explained variance ratio compared to the target explained variance (see the init vairables)
        # then add the eigenvector as column or the matrix_w above.
        # matrix_w will have the dimension of (784,1) initially, but each iteration the column will be added until 
        # the cumulative explained variance reaches the target explained variance.
        
        idxs = np.argsort(eig_pairs[0])[::-1]
        eigenvalues = eig_pairs[0][idxs]
        eigenvectors = eig_pairs[1].T
        eigenvectors = eigenvectors[idxs]
        for i in range(len(cum_var_exp)):
            if cum_var_exp[i] >= self.target_explained_variance:
                return matrix_w
            else:
                e_v = np.reshape(eigenvectors[i], (eigenvectors.shape[0],1))
                matrix_w = np.append(matrix_w, e_v, axis=1)
        return matrix_w

    def transform_data(self, X_std, matrix_w):
        """
        transform data to subspace using weight matrix
        :param X_std: standardized data
        :param matrix_w: weight matrix
        :return: data in the subspace
        """
        return X_std.dot(matrix_w)

    def fit(self, X):
        """    
        entry point to the transform data to k dimensions
        standardize and compute weight matrix to transform data.
        The fit functioin returns the transformed features. k is the number of features which cumulative 
        explained variance ratio meets the target_explained_variance.
        :param   m X n dimension: train samples
        :return  m X k dimension: subspace data. 
        """
    
        self.feature_size = X.shape[1]
        
        
        # Multisteps to appomplish the fit function- 16 pts
        # step 1. Standardize X to X_std using an appropriate function you implemented above.
        X_std = self.standardize(X)
        
        # step 2. get mean_vec and cov_mat from the appropriate functions from above implementations
        mean_vec = self.compute_mean_vector(X_std)
        cov_mat = self.compute_cov(X_std, mean_vec)
        
        # step 3. get eigenvalues and eigenvectors from the implemented function above.
        eig = self.compute_eigen_vector(cov_mat)

        # step 4. Sort both eig_vals and eig_vecs by descending order in eigenvalues. 
        # For example, the first 5 elements of the sorted eigenvalues would look like array([170.57702751, 112.84212831,  45.10927112,  40.75001861, 32.99731063])
        # and reorder the eigenvector list accordingly.
        # Make a list of tuple called eig_pairs
        # eig_pairs = [(170.577, the first eigenvector), (112.84, the second eigenvector), ...] (the length of this list is 784)
        # Each eigenvector has a dimension of (784,)
        # YOUR CODE HERE

        
        # step 5. get explained variance ratio and cumulated explained variance ratio using functions implemented above.
        # Use the variable names below.
        var_exp = self.compute_explained_variance(eig[0])
        cum_var_exp = self.cumulative_sum(var_exp)
        # YOUR CODE HERE
        
        # This step calculates the matrix_w
        matrix_w = self.compute_weight_matrix(eig_pairs=eig,cum_var_exp=cum_var_exp) 
        
        print(len(matrix_w),len(matrix_w[0]))
        return self.transform_data(X_std=X_std, matrix_w=matrix_w)


In [3]:
X_train = pickle.load(open('./data/fashionmnist/train_images.pkl','rb'))
y_train = pickle.load(open('./data/fashionmnist/train_image_labels.pkl','rb'))

X_train = X_train[:1500]
y_train = y_train[:1500]

In [5]:
# pca = PCA(target_explained_variance=0.99)
# std = pca.standardize(X_train)
# mv = pca.compute_mean_vector(std)
# cov_matrix = pca.compute_cov(std,mv)
# eigen = pca.compute_eigen_vector(cov_matrix)
# ex_var = pca.compute_explained_variance(eigen[0])
# cuml_sum = pca.cumulative_sum(ex_var)
# weighted_matrix = pca.compute_weight_matrix(eigen, cuml_sum)

# print(std.shape)
# print(mv.shape)
# print(cov_matrix.shape)
# print(eigen[0].shape)
# print(eigen[1].shape)
# print(cuml_sum.shape)

# print(len(eigen))
# print(eigan[1].shape)
# # print(matrix)
# # print(weighted_matrix.shape)

To Do: Complete helper functions above.

complete `fit()` to using all helper functions to find reduced dimension data.

Run PCA on *fashion mnist dataset* to reduce the dimension of the data.

fashion mnist data consists of samples with *784 dimensions*.

Report the reduced dimension $k$ for target explained variance of **0.99**

In [6]:
# X_train = pickle.load(open('./data/fashionmnist/train_images.pkl','rb'))
# y_train = pickle.load(open('./data/fashionmnist/train_image_labels.pkl','rb'))

# X_train = X_train[:1500]
# y_train = y_train[:1500]

In [7]:
pca_handler = PCA(target_explained_variance=0.99)
X_train_updated = pca_handler.fit(X_train)

784 410
